In [1]:
def traceback(path_matrix, start_pos, score_matrix):
    x,y = start_pos
    aligned_seqA = []
    aligned_seqB = []

    i = len(seqA)
    j = len(seqB)
    
    while x < i :
        aligned_seqA += seqA[i-1].lower()
        aligned_seqB += '-'
        i -= 1
    while y < j:
        aligned_seqA += '-'
        aligned_seqB += seqB[j-1].lower()
        j -= 1
    
    while path_matrix[x][y] != [0, 'NULL'] :
        d, direction = path_matrix[x][y][0], path_matrix[x][y][1]
        if direction == 'DIAG' :
            aligned_seqA.append(seqA[x - 1])     
            aligned_seqB.append(seqB[y - 1])
            x -= 1
            y -= 1
        elif direction == 'UP' : 
            for c in range(d):
                aligned_seqA.append(seqA[x - 1])
                aligned_seqB.append('-')             
                x -= 1
        elif direction == 'LEFT' :
            for c in range(d):
                aligned_seqA.append('-')            
                aligned_seqB.append(seqB[y - 1])
                y -= 1
    
    while x > 0:
        aligned_seqA += seqA[x-1].lower()
        aligned_seqB += '-'
        x -= 1
    while y > 0:
        aligned_seqA += '-'
        aligned_seqB += seqB[y-1].lower()
        y -= 1
    
    return ''.join(reversed(aligned_seqA)), ''.join(reversed(aligned_seqB))

In [2]:
def create_score_matrix(rows, cols):
    score_matrix = np.zeros((rows, cols))
    path_matrix = [[[0 , 'NULL'] for col in range(cols)] for row in range(rows)]  
    
    max_score = 0
    max_pos   = None    
    for i in range(1, rows): 
        for j in range(1, cols):
            score, antecedent = calc_score(score_matrix, i, j)
            if score > max_score:                  
                max_score = score
                max_pos   = (i, j)

            score_matrix[i][j],path_matrix[i][j] = score, antecedent           

    return score_matrix, max_pos, max_score, path_matrix  

In [3]:
def calc_score(score_matrix, x, y): 
    similarity = Substitution_score(match_score_matrix, x, y)
    
    same_row = [(score_matrix[x][y-l]-gap_penalty(l)) for l in range(1,y+1)]
    same_col = [(score_matrix[x-k][y]-gap_penalty(k)) for k in range(1,x+1)]
    
    up_score = max(same_col)
    left_score = max(same_row)
       
    diag_score = score_matrix[x - 1][y - 1] + similarity
    pos_max_up = first_pos_max(same_col)    
    pos_max_left = first_pos_max(same_row)                  

    score =  max(0, diag_score, up_score, left_score)  
    
    if score == diag_score:
        antecedent = [1, 'DIAG']
        return score, antecedent
    elif score == up_score:
        antecedent = [pos_max_up + 1, 'UP']
        return score, antecedent
    elif score == left_score: 
        antecedent = [pos_max_left + 1, 'LEFT']
        return score, antecedent 
    else:
        return score, [0, 'NULL']

In [4]:
def create_match_score_matrix(alphabet, match, mismatch):
    global match_score_matrix 
    match_score_matrix = np.zeros((len(alphabet), len(alphabet)))

    for i in range(len(alphabet)):
        for j in range(len(alphabet)):
            if alphabet[i] == alphabet[j] :
                match_score_matrix[i][j] = match
            elif alphabet[i] != alphabet[j]:
                match_score_matrix[i][j] = mismatch                
    return match_score_matrix

In [5]:
def Substitution_score(match_score_matrix, x, y):
    a_i = alphabet.index(seqA[x-1])
    b_j = alphabet.index(seqB[y-1])
    
    return match_score_matrix[a_i][b_j]

In [6]:
def first_pos_max(list1):
    maxi = max(list1)
    return [i for i, j in enumerate(list1) if j == maxi][0]

In [7]:
def gap_penalty(k): 
    return k*continue_gap_penalty + open_gap_penalty 

In [8]:
import numpy as np

seqA = 'tcccagttatgtcaGgggacacgagcatgcagagac'.upper()
seqB = 'aattgccgccgtcgttttcagcagttatgtcagatc'.upper()

match = 3
mismatch = -1
open_gap_penalty = 1
continue_gap_penalty  = 4

alphabet = 'ACGT' 

match_score_matrix = create_match_score_matrix(alphabet, match, mismatch)

rows = len(seqA) + 1
cols = len(seqB) + 1

score_matrix, start_pos, max_score, path_matrix = create_score_matrix(rows, cols)  

seqA_aligned, seqB_aligned = traceback(path_matrix, start_pos, score_matrix)

print(seqA_aligned)
print(seqB_aligned)

-----------------TC-CCAGTTATGTCAG---gggacacgagcatgcagagac
aattgccgccgtcgtttTCAGCAGTTATGTCAGatc---------------------


In [9]:
open_gap_penalty = 1
continue_gap_penalty  = 1

alphabet = 'ACGT' 

match_score_matrix = create_match_score_matrix(alphabet, match, mismatch)

rows = len(seqA) + 1
cols = len(seqB) + 1

score_matrix, start_pos, max_score, path_matrix = create_score_matrix(rows, cols)  

seqA_aligned, seqB_aligned = traceback(path_matrix, start_pos, score_matrix)

print(seqA_aligned)
print(seqB_aligned)

---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGA--gac
aatTGCCGCCGTCGT-TTTCAG----CA-G-TTATGTCAGAtc---
